In [10]:
# Import Statements
import pyrealsense2 as rs
import numpy as np
import math
import time
import cv2
import copy

In [11]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30) # depth stream
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30) # color stream
config.enable_stream(rs.stream.gyro) # gyro stream
config.enable_stream(rs.stream.accel) # accelerometer stream
align = rs.align(rs.stream.color) # align both streams to same pov

In [12]:
# Enable visualizer and filters for later use
colorizer = rs.colorizer()
spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 3)
hole_filling = rs.hole_filling_filter()
depth_to_disparity = rs.disparity_transform(True)
disparity_to_depth = rs.disparity_transform(False)

In [13]:
# Start streaming
profile = pipeline.start(config)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)
        
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        accel_frame = frames.first_or_default(rs.stream.accel)
        if accel_frame:
            accel_data = accel_frame.as_motion_frame().get_motion_data()
            print("\nAccel: ",accel_data.x,", ",accel_data.y,", ",accel_data.z,"\n")
            
        gyro_frame = frames.first_or_default(rs.stream.gyro)
        if gyro_frame:
            gyro_data = gyro_frame.as_motion_frame().get_motion_data()
            print("Gyro: ",gyro_data.x,", ",gyro_data.y,", ",gyro_data.z,"\n")
        
        # filter depth stream: depth2disparity -> spatial -> disparity2depth -> hole_filling
        depth_frame = depth_to_disparity.process(depth_frame)
        depth_frame = spatial.process(depth_frame)
        depth_frame = disparity_to_depth.process(depth_frame)
        depth_frame = hole_filling.process(depth_frame)
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # Stack both images horizontally
        images = np.hstack((color_image, depth_image))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break

finally:
    # Stop streaming
    pipeline.stop()


Accel:  1.81423020362854 ,  -4.37376594543457 ,  8.982891082763672 

Gyro:  0.0872664600610733 ,  -0.022689279168844223 ,  -0.008726646192371845 


Accel:  1.7553902864456177 ,  -4.4620256423950195 ,  8.845598220825195 

Gyro:  -0.006981316953897476 ,  -0.010471975430846214 ,  -0.003490658476948738 


Accel:  1.8436501026153564 ,  -4.37376594543457 ,  8.94366455078125 

Gyro:  0.04712389037013054 ,  0.006981316953897476 ,  -0.001745329238474369 


Accel:  1.7651969194412231 ,  -4.354152679443359 ,  9.012310981750488 

Gyro:  -0.04363323003053665 ,  -0.010471975430846214 ,  0.0 


Accel:  1.7848103046417236 ,  -4.422799110412598 ,  8.884824752807617 

Gyro:  -0.006981316953897476 ,  0.010471975430846214 ,  0.003490658476948738 


Accel:  1.81423020362854 ,  -4.354152679443359 ,  8.982891082763672 

Gyro:  0.04188790172338486 ,  -0.013962633907794952 ,  -0.005235987715423107 


Accel:  1.7651969194412231 ,  -4.422799110412598 ,  8.8554048538208 

Gyro:  0.0 ,  0.001745329238474369 ,  0.


Accel:  1.81423020362854 ,  -4.334539413452148 ,  9.0319242477417 

Gyro:  -0.010471975430846214 ,  0.010471975430846214 ,  0.006981316953897476 


Accel:  1.7848103046417236 ,  -4.324732780456543 ,  8.982891082763672 

Gyro:  -0.04188790172338486 ,  -0.005235987715423107 ,  0.006981316953897476 


Accel:  1.7553902864456177 ,  -4.403185844421387 ,  8.786758422851562 

Gyro:  0.07504915446043015 ,  -0.001745329238474369 ,  -0.005235987715423107 


Accel:  1.81423020362854 ,  -4.354152679443359 ,  9.041730880737305 

Gyro:  -0.0296705961227417 ,  0.005235987715423107 ,  0.001745329238474369 


Accel:  1.7848103046417236 ,  -4.354152679443359 ,  8.94366455078125 

Gyro:  -0.05061454698443413 ,  0.001745329238474369 ,  0.005235987715423107 


Accel:  1.7651969194412231 ,  -4.403185844421387 ,  8.884824752807617 

Gyro:  -0.019198622554540634 ,  0.0296705961227417 ,  0.001745329238474369 


Accel:  1.6867437362670898 ,  -4.422799110412598 ,  8.982891082763672 

Gyro:  0.06108652427792549 


Accel:  1.6867437362670898 ,  -4.324732780456543 ,  8.845598220825195 

Gyro:  0.03839724510908127 ,  -0.04712389037013054 ,  -0.01745329238474369 


Accel:  1.863263487815857 ,  -4.442412376403809 ,  9.071150779724121 

Gyro:  -0.022689279168844223 ,  0.031415924429893494 ,  0.005235987715423107 


Accel:  1.7848103046417236 ,  -4.471832275390625 ,  9.041730880737305 

Gyro:  0.005235987715423107 ,  -0.010471975430846214 ,  -0.005235987715423107 


Accel:  1.7848103046417236 ,  -4.324732780456543 ,  8.845598220825195 

Gyro:  -0.10471975803375244 ,  0.010471975430846214 ,  0.0 


Accel:  1.81423020362854 ,  -4.511058807373047 ,  8.8554048538208 

Gyro:  0.11344639956951141 ,  0.0 ,  -0.006981316953897476 


Accel:  1.863263487815857 ,  -4.305119037628174 ,  9.169217109680176 

Gyro:  -0.05235987901687622 ,  0.005235987715423107 ,  0.003490658476948738 


Accel:  1.7357770204544067 ,  -4.422799110412598 ,  8.816178321838379 

Gyro:  -0.07679449021816254 ,  0.0296705961227417 ,  0.0174

Accel:  2.098623037338257 ,  -4.71699857711792 ,  8.894631385803223 

Gyro:  0.2321287840604782 ,  -0.010471975430846214 ,  0.03490658476948738 


Accel:  2.098623037338257 ,  -4.197246074676514 ,  8.482751846313477 

Gyro:  0.4136430323123932 ,  -0.16755160689353943 ,  -0.06457718461751938 


Accel:  1.3729310035705566 ,  -4.040339469909668 ,  8.8554048538208 

Gyro:  0.404916375875473 ,  0.02094395086169243 ,  0.07853981852531433 


Accel:  2.000556468963623 ,  -5.040617942810059 ,  9.110377311706543 

Gyro:  1.0192723274230957 ,  -0.2844886779785156 ,  0.20943951606750488 


Accel:  1.235637903213501 ,  -3.4911673069000244 ,  9.21825122833252 

Gyro:  1.0890854597091675 ,  -0.30717796087265015 ,  0.08377580344676971 


Accel:  2.098623037338257 ,  -2.922381639480591 ,  10.257755279541016 

Gyro:  0.6981316804885864 ,  0.1186823844909668 ,  0.022689279168844223 


Accel:  1.8436501026153564 ,  -3.0989012718200684 ,  9.316317558288574 

Gyro:  0.7714354991912842 ,  -0.0645771846175193


Accel:  -5.707469940185547 ,  -6.717555046081543 ,  1.235637903213501 

Gyro:  0.25307273864746094 ,  0.28274333477020264 ,  0.8761552572250366 


Accel:  -7.217694282531738 ,  -10.365629196166992 ,  2.677215337753296 

Gyro:  0.15882496535778046 ,  -1.3404128551483154 ,  -0.7749261856079102 


Accel:  -6.796008110046387 ,  -7.845319747924805 ,  1.402350902557373 

Gyro:  0.3752457797527313 ,  -0.9721483588218689 ,  -0.7539822459220886 


Accel:  -5.717276573181152 ,  -7.629573345184326 ,  1.9221034049987793 

Gyro:  -0.42236968874931335 ,  -0.27576202154159546 ,  -0.11519172787666321 


Accel:  -6.854848384857178 ,  -6.3350958824157715 ,  3.0596747398376465 

Gyro:  0.010471975430846214 ,  0.3001966178417206 ,  -0.7504915595054626 


Accel:  -5.933022975921631 ,  -6.590068817138672 ,  2.098623037338257 

Gyro:  -0.404916375875473 ,  0.48869219422340393 ,  -1.3875367641448975 


Accel:  -4.471832275390625 ,  -6.805814743041992 ,  1.833843469619751 

Gyro:  -0.1500983089208603 ,  0.448


Accel:  -0.5982056260108948 ,  -9.110377311706543 ,  2.3634026050567627 

Gyro:  0.593411922454834 ,  1.141445279121399 ,  -0.36651915311813354 


Accel:  -0.7747253179550171 ,  -9.120183944702148 ,  1.5592572689056396 

Gyro:  0.6143559217453003 ,  1.624901533126831 ,  -0.34208452701568604 


Accel:  -1.0885381698608398 ,  -8.786758422851562 ,  0.9512450098991394 

Gyro:  0.44854962825775146 ,  1.4311699867248535 ,  -0.2879793345928192 


Accel:  -0.509945809841156 ,  -9.64974308013916 ,  1.1767979860305786 

Gyro:  0.18325957655906677 ,  1.2653636932373047 ,  -0.359537810087204 


Accel:  -0.40207263827323914 ,  -9.81645679473877 ,  1.2846710681915283 

Gyro:  -0.005235987715423107 ,  1.0541788339614868 ,  -0.3281219005584717 


Accel:  -0.8041452765464783 ,  -10.444082260131836 ,  -0.029419949278235435 

Gyro:  -0.13089969754219055 ,  0.5253440737724304 ,  -0.42935100197792053 


Accel:  -0.34323275089263916 ,  -10.316595077514648 ,  0.8727918267250061 

Gyro:  -0.6091198921203613 

Accel:  -1.9515233039855957 ,  -9.75761604309082 ,  -0.029419949278235435 

Gyro:  -0.04712389037013054 ,  0.0296705961227417 ,  0.06108652427792549 


Accel:  -2.0692031383514404 ,  -9.394770622253418 ,  0.029419949278235435 

Gyro:  -0.0872664600610733 ,  0.022689279168844223 ,  0.013962633907794952 


Accel:  -2.0495898723602295 ,  -9.64974308013916 ,  -0.13729310035705566 

Gyro:  -0.04363323003053665 ,  0.02094395086169243 ,  0.02094395086169243 


Accel:  -2.2359161376953125 ,  -9.610516548156738 ,  -0.22555294632911682 

Gyro:  -0.013962633907794952 ,  -0.02094395086169243 ,  0.03490658476948738 


Accel:  -1.8436501026153564 ,  -9.581096649169922 ,  -0.22555294632911682 

Gyro:  -0.0872664600610733 ,  -0.027925267815589905 ,  0.07155849784612656 


Accel:  -2.206496238708496 ,  -9.75761604309082 ,  -0.009806649759411812 

Gyro:  0.005235987715423107 ,  0.02094395086169243 ,  0.05235987901687622 


Accel:  -2.41243577003479 ,  -9.443803787231445 ,  -0.13729310035705566 

Gyro:  



Accel:  -2.1770763397216797 ,  -6.992141246795654 ,  -3.0400614738464355 

Gyro:  -0.45378559827804565 ,  2.64591908454895 ,  -0.3228859007358551 


Accel:  -2.265336036682129 ,  -9.963556289672852 ,  -5.579983711242676 

Gyro:  -0.16231562197208405 ,  2.043780565261841 ,  -0.3892084062099457 


Accel:  -1.402350902557373 ,  -7.266727447509766 ,  -3.216581106185913 

Gyro:  -0.5445427298545837 ,  1.53239905834198 ,  -0.24609142541885376 


Accel:  -1.7651969194412231 ,  -8.492558479309082 ,  -5.177911281585693 

Gyro:  -0.4380776286125183 ,  1.2653636932373047 ,  -0.7539822459220886 


Accel:  -1.0591181516647339 ,  -10.03220272064209 ,  -4.540478706359863 

Gyro:  -0.2879793345928192 ,  0.9948376417160034 ,  -0.11344639956951141 


Accel:  -0.6962721347808838 ,  -8.316039085388184 ,  -3.961886405944824 

Gyro:  -0.21467550098896027 ,  1.0559241771697998 ,  -0.21991148591041565 


Accel:  0.19613298773765564 ,  -9.708582878112793 ,  -4.422799110412598 

Gyro:  0.09424778074026108 ,  